# 🎯 Objetivo
Este notebook tem como objetivo complementar o nosso notebook principal, focando especificamente nas respostas às perguntas formuladas no início da Etapa 1. Optamos por estruturá-lo separadamente para garantir uma melhor organização e clareza na apresentação das conclusões.

## 📦 Instalação da Biblioteca `nbimporter`

Para importar variáveis, funções ou células de um notebook Jupyter para outro, utilizamos a biblioteca `nbimporter`. 

🔧 **Para instalá-la, execute o seguinte comando no terminal ou em uma célula do Jupyter:**

```bash
pip install nbimporter


In [3]:
import nbimporter
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import ttest_ind
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')


<a id="perguntas"></a>
# ❓ Perguntas e Respostas

A seguir, apresentamos cada pergunta acompanhada de sua resposta final, com base nas análises realizadas ao longo das Etapas 1 a 5 do projeto. Quando aplicável, também incluímos referências às etapas, gráficos, tabelas e demais visualizações que serviram de base para a resposta.

Algumas questões foram abordadas em diferentes momentos do projeto — nesses casos, consolidamos as evidências para oferecer uma visão mais completa.

<a id="1"></a>
### 1 - Existe correlação entre o PIB per capita de uma região e a taxa de mortalidade infantil?

#### Com base na análise comparativa entre a distribuição regional dos óbitos infantis e a do PIB per capita, é possível identificar uma correlação negativa entre essas duas variáveis. Os gráficos mostram que regiões com maior participação no PIB per capita, como o Sul, Sudeste e Centro-Oeste, concentram proporcionalmente menos óbitos infantis ao longo dos anos analisados. Em contrapartida, o Nordeste, que apresenta a menor fatia de participação no PIB per capita lidera na proporção de mortalidade infantil.


In [4]:

# 1. Preparação dos dados
df_obitos = pd.read_csv('dataFrame/BrStatsAtualizada.csv', sep=';', low_memory=False)
df_pib = pd.read_csv('pib_per_capita/pib_per_capita_regioes.csv')

# Converter dados numéricos
df_obitos['NrObitosInfantis'] = df_obitos['NrObitosInfantis'].astype(str).str.replace('.','').str.replace(',','.').astype(float)
df_obitos['Ano'] = df_obitos['Ano'].astype(str)
df_pib['Ano'] = df_pib['Ano'].astype(str)

# Agregar e juntar dados
df_agregado = df_obitos.groupby(['Ano', 'Regiao'])['NrObitosInfantis'].sum().reset_index()
df_final = pd.merge(df_agregado, df_pib, on=['Ano', 'Regiao'])

# 2. Criar gráfico de área percentual para Óbitos
fig_obitos = px.area(
    df_final,
    x='Ano',
    y='NrObitosInfantis',
    color='Regiao',
    title='<b>Distribuição Percentual dos Óbitos Infantis por Região</b>',
    labels={'NrObitosInfantis': 'Participação % nos Óbitos', 'Ano': 'Ano'},
    groupnorm='percent',  # Normaliza para 100%
    color_discrete_sequence=px.colors.qualitative.Pastel
)

# 3. Criar gráfico de área percentual para PIB
fig_pib = px.area(
    df_final,
    x='Ano',
    y='PIB_per_capita',
    color='Regiao',
    title='<b>Distribuição Percentual do PIB per capita por Região</b>',
    labels={'PIB_per_capita': 'Participação % no PIB', 'Ano': 'Ano'},
    groupnorm='percent',  # Normaliza para 100%
    color_discrete_sequence=px.colors.qualitative.Pastel
)

# 4. Ajustar layouts
for fig in [fig_obitos, fig_pib]:
    fig.update_layout(
        height=400,
        width=800,
        hovermode='x unified',
        yaxis=dict(ticksuffix='%'),
        legend=dict(orientation='h', yanchor='bottom', y=1.02),
        margin=dict(l=50, r=50, t=80, b=50)
    )
    fig.update_traces(
        hovertemplate='<b>%{color}</b><br>Ano: %{x}<br>Participação: %{y:.1f}%'
    )

# 5. Exibir gráficos lado a lado
from plotly.subplots import make_subplots

final_fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Distribuição dos Óbitos Infantis por Região',
    'Distribuição do PIB per capita por Região'
))

for trace in fig_obitos.data:
    final_fig.add_trace(trace, row=1, col=1)
    
for trace in fig_pib.data:
    final_fig.add_trace(trace, row=1, col=2)

final_fig.update_layout(
    height=450,
    width=1000,
    showlegend=True,
    title_text='<b>Comparação da Distribuição Regional: Óbitos vs PIB (2016-2021)</b>',
    title_x=0.5
)

final_fig.show()

### Ademais, ao observar as regras de associação,a Regra 1 confirma a expectativa geral: regiões com PIB per capita alto tendem a apresentar mortalidade infantil baixa, o que é coerente tanto com a lógica socioeconômica quanto com a visualização dos gráficos, nos quais regiões como Sul, Sudeste e Centro-Oeste (com maior PIB) possuem menores proporções de óbitos. A confiança alta (86,01%) reforça a consistência dessa associação. No entanto, o lift inferior a 1 sugere que, embora exista relação, ela não é tão forte quanto se esperaria, o que indica que outros fatores podem estar atuando simultaneamente (como políticas públicas, estrutura de saúde e fatores demográficos).

### As Regras 2 e 3, por outro lado, são contraintuitivas. Ambas mostram que, mesmo com PIB per capita baixo, a mortalidade infantil também tende a ser baixa, com destaque para a Regra 3, cuja confiança é altíssima (94,23%). Isso levanta uma hipótese importante: em determinadas regiões de menor PIB, mecanismos compensatórios — como programas sociais, atenção básica à saúde, saneamento mínimo ou iniciativas federais — podem estar sendo eficazes em manter baixos os índices de mortalidade infantil.

### Do ponto de vista gráfico, essa hipótese é sustentada: o Nordeste, por exemplo, tem baixa participação no PIB per capita, mas ainda assim representa uma parte importante das regiões com mortalidade infantil relativamente controlada, sem grandes aumentos ao longo dos anos. Isso se alinha com a observação de que o PIB por si só não determina os resultados de saúde infantil, embora ajude a explicá-los em parte

In [13]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Carregar o arquivo
df = pd.read_csv("dataFrame/BrStatsAtualizada.csv", sep=";", low_memory=False)

# Converter colunas numéricas com vírgula para float
for col in ['PIB', 'Populacao', 'NrNascimentos', 'NrObitosInfantis']:
    df[col] = df[col].astype(str).str.replace(r'\.', '', regex=True).str.replace(',', '.').astype(float)

# Calcular o PIB per capita
df['PIB_per_capita'] = df['PIB'] / df['Populacao']

# Calcular a taxa de mortalidade infantil (óbitos por 1000 nascimentos)
df['TaxaMortalidadeInfantil'] = (df['NrObitosInfantis'] / df['NrNascimentos']) * 1000


# 📊 Classificar PIB per capita como 'Alto' ou 'Baixo'
media_pib_per_capita = df['PIB_per_capita'].mean()
df['PIB_per_capita_cat'] = df['PIB_per_capita'].apply(lambda x: 'Alto' if x > media_pib_per_capita else 'Baixo')

# 📊 Classificar mortalidade infantil como 'Alta' ou 'Baixa'
media_mortalidade = df['TaxaMortalidadeInfantil'].mean()
df['MortalidadeInfantil_cat'] = df['TaxaMortalidadeInfantil'].apply(lambda x: 'Alta' if x > media_mortalidade else 'Baixa')






# Criar DataFrame para Apriori com variáveis categóricas binárias
df_apriori = pd.get_dummies(df[['PIB_per_capita_cat', 'MortalidadeInfantil_cat']])
frequent_itemsets = apriori(df_apriori, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.6)

# Filtrar pela regra: Se PIB_per_capita_Alto então MortalidadeInfantil_Baixa
regras_filtradas = rules[
    (rules['antecedents'].astype(str).str.contains('PIB_per_capita_cat_Alto')) &
    (rules['consequents'].astype(str).str.contains('MortalidadeInfantil_cat_Baixa'))
]
for index, row in rules.iterrows():
    print(f"Regra {index + 1}: {row['antecedents']} => {row['consequents']}")
    print(f"Suporte: {row['support']:.4f}, Confiança: {row['confidence']:.4f}, Lift: {row['lift']:.4f}")
    print('-' * 80)


Regra 1: frozenset({'PIB_per_capita_cat_Alto'}) => frozenset({'MortalidadeInfantil_cat_Baixa'})
Suporte: 0.2822, Confiança: 0.8601, Lift: 0.9396
--------------------------------------------------------------------------------
Regra 2: frozenset({'MortalidadeInfantil_cat_Baixa'}) => frozenset({'PIB_per_capita_cat_Baixo'})
Suporte: 0.6332, Confiança: 0.6917, Lift: 1.0295
--------------------------------------------------------------------------------
Regra 3: frozenset({'PIB_per_capita_cat_Baixo'}) => frozenset({'MortalidadeInfantil_cat_Baixa'})
Suporte: 0.6332, Confiança: 0.9423, Lift: 1.0295
--------------------------------------------------------------------------------


### Portanto,sim, há uma correlação entre PIB per capita e mortalidade infantil, mas ela não é direta nem uniforme. A Regra 1 mostra uma tendência esperada (PIB alto → mortalidade baixa), mas o lift fraco e os resultados de Regras 2 e 3 revelam que fatores estruturais e políticas públicas podem estar quebrando a lógica clássica da desigualdade econômica em algumas regiões. Isso reforça a ideia de que a redução da mortalidade infantil não depende apenas do crescimento econômico, mas de como os recursos são aplicados e das políticas adotadas para garantir equidade no acesso à saúde.

<a id="2"></a>
### 2 - Regiões com maior produção agrícola têm maior PIB per capita?

#### Não, necessariamente. Embora algumas regiões com alta produção agrícola também tenham PIB per capita elevado, isso não é uma regra geral. A análise mostra que a produção agrícola por si só não garante maior desenvolvimento econômico. O que realmente faz diferença é a capacidade de agregar valor à produção, por meio de industrialização, logística eficiente e diversificação econômica.

In [ ]:

df_agricola = pd.read_csv('dataFrame/BrStatsAtualizada.csv', sep=';', low_memory=False)
df_pib_regioes = pd.read_csv('pib_per_capita/pib_per_capita_regioes.csv')  # Dados já agregados por região

# Converter colunas numéricas (tratando formato brasileiro)
if 'VlProducaoAgricola' in df_agricola.columns:
    df_agricola['VlProducaoAgricola'] = df_agricola['VlProducaoAgricola'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)

# Agregar produção agrícola por região e ano (soma)
df_agregado_regiao = df_agricola.groupby(['Ano', 'Regiao'])['VlProducaoAgricola'].sum().reset_index()

# Contar número de municípios por região/ano (para o tamanho dos pontos)
qtd_municipios = df_agricola.groupby(['Ano', 'Regiao'])['Municipio'].nunique().reset_index()
qtd_municipios.rename(columns={'Municipio': 'QtdMunicipios'}, inplace=True)

# Juntar todos os dados (usando a tabela de PIB por regiões)
df_final = pd.merge(df_agregado_regiao, df_pib_regioes, on=['Ano', 'Regiao'])
df_final = pd.merge(df_final, qtd_municipios, on=['Ano', 'Regiao'])

# Converter Ano para string para a animação
df_final['Ano'] = df_final['Ano'].astype(str)

# Criar o gráfico
fig = px.scatter(
    df_final,
    x='PIB_per_capita',  # Coluna da sua tabela de regiões
    y='VlProducaoAgricola',
    color='Regiao',
    size='QtdMunicipios',
    animation_frame='Ano',
    range_x=[0, df_final['PIB_per_capita'].max() * 1.1],
    range_y=[0, df_final['VlProducaoAgricola'].max() * 1.1],
    title='<b>Relação entre PIB per capita e Produção Agrícola por Região (2016-2021)</b>',
    labels={
        'PIB_per_capita': 'PIB per capita (R$)',
        'VlProducaoAgricola': 'Valor da Produção Agrícola (R$)',
        'QtdMunicipios': 'Nº de Municípios',
        'Regiao': 'Região'
    },
    hover_data=['Regiao', 'Ano'],
    height=600,
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_layout(
    font=dict(family="Arial", size=12),
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font_size=20,
    legend_title_text='Região',
    margin=dict(l=40, r=40, t=80, b=40),
    yaxis=dict(gridcolor='lightgrey'),
    xaxis=dict(gridcolor='lightgrey'),
    hovermode='closest'
)

fig.update_traces(
    marker=dict(opacity=0.85, line=dict(width=1, color='DarkSlateGrey'))
)

# Configurações da animação
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 500

fig.show()

<a id="3"></a>
### 3 - Cidades com maior área cultivada têm menor densidade populacional? Isso afeta o desenvolvimento econômico?

<a id="4"></a>
### 4 - Cidades mais populosas têm mais empregos, mas piores indicadores de saúde?

<a id="5"></a>
### 5 - Estados com maior volume de exportações têm maior PIB per capita?
#### Não necessariamente. Com base na análise do gráfico de Exportações vs PIB per capita por Estado, é possível perceber que um maior volume de exportações não garante, por si só, um maior PIB per capita. O estado de São Paulo, por exemplo, apresenta um alto volume de exportações, mas não lidera o ranking de PIB per capita. Por outro lado, o Distrito Federal se destaca entre os estados com maior PIB per capita, mesmo com um volume exportador relativamente baixo, o que indica que sua economia está fortemente baseada em setores como administração pública e serviços. Esses exemplos mostram que o desenvolvimento econômico regional depende de múltiplos fatores além do comércio exterior. Entre eles, destacam-se a composição setorial da economia, o nível de diversificação produtiva, a infraestrutura institucional e a presença de políticas públicas que impactam diretamente o rendimento per capita da população.

🔍 Dica: Para visualizar o PIB per capita dos estados de SP e AM, clique na legenda da cor azul (Exportações) no gráfico. Isso ocultará temporariamente as barras de exportações e permitirá ver com mais clareza as barras de PIB per capita desses estados.

In [ ]:
df = pd.read_csv("dataFrame/BrStatsAtualizada.csv", sep=";", dtype={"Exportacoes_R$": str})
df.columns = df.columns.str.strip()

# Carregar o dataset de PIB per capita por estado
df_pib_estado = pd.read_csv("pib_per_capita/pib_per_capita_estados.csv")
df_pib_estado["Ano"] = df_pib_estado["Ano"].astype(str)

for col in ["Exportacoes_R$", "QtEmpresas", "PessoalOcupado"]:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(".", "", regex=False).str.replace(",", ".", regex=False),errors="coerce")
    
df["Ano"] = df["Ano"].astype(str)
df_exportacoes = df.groupby(["Ano", "UF"])["Exportacoes_R$"].sum().reset_index()
df_final = pd.merge(df_exportacoes, df_pib_estado, on=["Ano", "UF"])
fig = go.Figure()
anos = sorted(df_final["Ano"].unique())
buttons = []

for i, ano in enumerate(anos):
    df_ano = df_final[df_final["Ano"] == ano].sort_values("PIB_per_capita", ascending=False)
    
    fig.add_trace(go.Bar(
        x=df_ano["UF"],
        y=df_ano["Exportacoes_R$"],
        name="Exportações (R$)",
        marker_color="#1f77b4",
        yaxis="y2",
        visible=(i==0)
    ))

    fig.add_trace(go.Bar(
        x=df_ano["UF"],
        y=df_ano["PIB_per_capita"],
        name="PIB per capita (R$)",
        marker_color="#ff7f0e",
        visible=(i==0)
    ))

    buttons.append(dict(
        label=str(ano),
        method="update",
        args=[{"visible": [False]*len(anos)*2},
              {"title": f"Exportações vs PIB per capita por Estado - {ano}"}]
    ))
    buttons[-1]["args"][0]["visible"][i*2] = True
    buttons[-1]["args"][0]["visible"][i*2 + 1] = True

fig.update_layout(
    title="Exportações vs PIB per capita por Estado",
    xaxis=dict(title="Estados", tickangle=45),
    yaxis=dict(title="PIB per capita (R$)", side="left"),
    yaxis2=dict(
        title="Exportações (R$)",
        overlaying="y",
        side="right"
        
    ),
    updatemenus=[{
        "buttons": buttons,
        "direction": "down",
        "showactive": True,
        "x": 1.0,
        "y": 1.15
    }],
    barmode="group",
    plot_bgcolor="white",
    height=600,
    width=1000,
    legend=dict(x=1.1, y=1)
)

fig.show()



<a id="6"></a>
### 6 - Cidades com maior produção agrícola têm melhores ou piores indicadores de saúde infantil?

####  Cidades com maior produção agrícola tendem a ter melhores indicadores de saúde infantil, conforme indicado pelo gráfico. Observa-se que as regiões com alta produção agrícola,em especial o Sul (laranja),concentram-se no extremo esquerdo do eixo X, o que representa menores taxas de mortalidade infantil (entre 20 e 24 por mil nascidos vivos). Por outro lado, regiões com baixa produção agrícola, como o Norte (verde) e partes do Centro-Oeste (azul), estão associadas a índices mais altos de mortalidade infantil, variando entre 35 e 44. Portanto, a relação mostrada no gráfico indica que quanto maior a produção agrícola, menor tende a ser a mortalidade infantil, sugerindo uma correlação positiva entre desenvolvimento agrícola e melhores condições de saúde infantil.

In [ ]:


df = pd.read_csv('dataFrame/BrStatsAtualizada.csv', sep=';', low_memory=False)

df['NrObitosInfantis'] = pd.to_numeric(
    df['NrObitosInfantis'].astype(str).str.replace('.', '').str.replace(',', '.'), 
    errors='coerce'
)
df['NrNascimentos'] = pd.to_numeric(
    df['NrNascimentos'].astype(str).str.replace('.', '').str.replace(',', '.'), 
    errors='coerce'
)
df['VlProducaoAgricola'] = pd.to_numeric(
    df['VlProducaoAgricola'].astype(str).str.replace('.', '').str.replace(',', '.'), 
    errors='coerce'
)

df['MortalidadeInfantil'] = (df['NrObitosInfantis'] / df['NrNascimentos']) * 1000

df_regiao = df.groupby(['Regiao', 'Ano']).agg({
    'NrObitosInfantis': 'sum',
    'NrNascimentos': 'sum',
    'VlProducaoAgricola': 'sum'
}).reset_index()

df_regiao['MortalidadeInfantil'] = (df_regiao['NrObitosInfantis'] / df_regiao['NrNascimentos']) * 1000

df_regiao = df_regiao.dropna(subset=['MortalidadeInfantil', 'VlProducaoAgricola'])
df_regiao = df_regiao[df_regiao['MortalidadeInfantil'] != float('inf')]

df_regiao['VlProducaoAgricola_M'] = df_regiao['VlProducaoAgricola'] / 1_000_000

fig = px.scatter(
    df_regiao,
    x='MortalidadeInfantil',
    y='VlProducaoAgricola_M',  
    color='Regiao',
    size='VlProducaoAgricola_M',
    size_max=30,  
    hover_name='Regiao',
    hover_data=['Ano', 'VlProducaoAgricola'],
    labels={
        'MortalidadeInfantil': 'Mortalidade Infantil (por 1000 nascidos vivos)',
        'VlProducaoAgricola_M': 'Produção Agrícola (em milhões de R$)',
        'Regiao': 'Região',
        'Ano': 'Ano'
    },
    title='<b>Relação entre Produção Agrícola e Mortalidade Infantil (por Região/Ano)</b>',
)

fig.update_layout(
    height=600,
    width=900,
    xaxis_title='Mortalidade Infantil (por 1000 nascidos vivos)',
    yaxis_title='Produção Agrícola (em milhões de R$)',
    legend_title='Região',
    hovermode='closest',
    xaxis=dict(
        tickformat=".2f" 


))

fig.show()

<a id="7"></a>
### 7 - Regiões com maior produção pecuária têm maior PIB per capita?

<a id="8"></a>
### 8 - Cidades com maior densidade populacional têm maior volume de exportações?
#### Sim. Embora o modelo de regressão linear tenha apresentado baixo poder preditivo (com R² de apenas 6,39%), os resultados apontam que a densidade populacional possui um coeficiente positivo elevado, indicando uma tendência de que estados mais densamente povoados tendem a exportar mais. Ou seja, mesmo com limitações no ajuste global do modelo, há indícios de que a densidade populacional exerce alguma influência positiva sobre o volume de exportações. Esse padrão sugere que fatores como concentração urbana, infraestrutura e logística podem contribuir para o desempenho exportador dos estados.

In [ ]:
# Carregar o dataset
df = pd.read_csv("dataFrame/BrStatsAtualizada.csv", sep=";")

# Converter as colunas relevantes de string para float (substituir vírgula por ponto)
for col in ["Exportacoes_R$", "povoamento", "QtEmpresas"]:
    df[col] = df[col].astype(str).str.replace(",", ".").astype(float)

# Selecionar as variáveis relevantes
df_model = df[["Exportacoes_R$", "povoamento", "QtEmpresas"]].copy()

# Remover registros com valores faltantes
df_model.dropna(inplace=True)

# Definir variáveis explicativas (X) e variável alvo (y)
X = df_model[["povoamento", "QtEmpresas"]]
y = df_model["Exportacoes_R$"]

# Padronizar as variáveis explicativas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir em conjuntos de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# Criar o modelo de regressão linear
model = LinearRegression()

# Treinar o modelo com os dados de treino
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Obter coeficientes e intercepto
coeficientes = model.coef_
intercepto = model.intercept_

# Calcular métricas de avaliação
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Mostrar os coeficientes com os nomes das variáveis
for nome, coef in zip(["Densidade Populacional (povoamento)", "Número de Empresas (QtEmpresas)"], coeficientes):
    print(f"{nome}: {coef:.4f}")

print(f"\nIntercepto: {intercepto:.2f}")
print(f"Erro Quadrático Médio (MSE): {mse:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2%}")


Densidade Populacional (povoamento): 602281.6500
Número de Empresas (QtEmpresas): -647206.8851

Intercepto: 461016291.49
Erro Quadrático Médio (MSE): 12273326486112360448.00
Coeficiente de Determinação (R²): 6.39%


<a id="9"></a>
### 9 - Estados com maior informalidade têm menor PIB?


#### Não necessariamente. Os dois gráficos sugerem uma tendência geral de que estados com maior número de empregos informais tendem a apresentar PIB per capita mais baixo, o que indicaria uma relação entre informalidade e menor desenvolvimento econômico. No entanto, essa relação não é absoluta.

#### No primeiro gráfico, com todos os estados (inclusive o DF), o Distrito Federal aparece como um ponto fora da curva, com um PIB per capita muito alto, mesmo com um número elevado de empregos informais, o que quebra a tendência esperada. Já no segundo gráfico, ao removermos o DF, conseguimos observar mais claramente a tendência negativa entre informalidade e PIB, mas ainda existem exceções, como o Mato Grosso (MT), que apresenta um PIB per capita elevado apesar de também ter uma informalidade significativa.

#### Portanto, embora exista uma associação aparente entre alta informalidade e menor PIB per capita, os dados mostram que essa relação não é uma regra fixa, e pode ser influenciada por outros fatores, como características econômicas regionais e estrutura produtiva.

In [11]:

# Carregar os dados
df = pd.read_csv('dataFrame/BrStatsAtualizada.csv', sep=';', low_memory=False)
df_pib = pd.read_csv('pib_per_capita/pib_per_capita_estados.csv')


for col in ['PessoalOcupado', 'PessoalAssalariado']:
    df[col] = df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')  

# Calcular empregos informais
df['Empregos_Informais'] = df['PessoalOcupado'] - df['PessoalAssalariado']

# Agrupar por estado e calcular média entre os anos
df_estado = df.groupby('UF')[['Empregos_Informais']].mean().round(0).astype(int).reset_index()


# Adicionar o PIB per capita médio de cada estado
pib_estado = df_pib.groupby('UF')['PIB_per_capita'].mean().reset_index()
df_estado = pd.merge(df_estado, pib_estado, on='UF')

fig = px.scatter(
    df_estado,
    x='Empregos_Informais',
    y='PIB_per_capita',
    text='UF',
    title='Relação entre Empregos Informais e PIB per capita por Estado (com escala log)',
    labels={
        'Empregos_Informais': 'Empregos Informais (média 2016–2021)',
        'PIB_per_capita': 'PIB per capita médio (R$)'
    },
    log_x=True  
)

fig.update_traces(textposition='top center')
fig.show()


In [12]:
# Identificar o estado com o maior número de "Empregos Informais"
df_estado_max_empregos = df_estado[df_estado['Empregos_Informais'] == df_estado['Empregos_Informais'].max()]
estado_max_empregos = df_estado_max_empregos['UF'].values[0]
#print(f'O estado com o maior número de empregos informais é: {estado_max_empregos}')

# Gráfico 2: Sem o estado com o maior número de "Empregos Informais"
df_estado_sem_max_empregos = df_estado[df_estado['UF'] != estado_max_empregos]

fig_sem_max_empregos = px.scatter(
    df_estado_sem_max_empregos,
    x='Empregos_Informais',
    y='PIB_per_capita',
    text='UF',

    title=f'Relação entre Empregos Informais e PIB per capita por Estado (sem {estado_max_empregos})',
    labels={
        'Empregos_Informais': 'Empregos Informais (média 2016–2021)',
        'PIB_per_capita': 'PIB per capita médio (R$)'
    },
    log_x=True  # Escala logarítmica
)

fig_sem_max_empregos.update_traces(textposition='top center')
fig_sem_max_empregos.show()
